In [1]:
From Coq Require Import ZArith Reals Psatz.
From Flocq Require Import Core Plus_error.

Open Scope R_scope.

Cell evaluated. 
 

 
 
 Cell rolled back. 
 

 
 
 
 Rollback cell 
 

 
 
 Auto rollback

In [93]:
Definition leapfrog_1 xn vn h F := 
    let vnp12 := vn + 0.5*  h* F xn in
    let xnp1  := xn + h* vn in
    pair xnp1 vnp12.
    
Definition jacob_leapfrog_1 xn h (dFdx: R -> R) :=
    let s1 := 1.0 in
    let s2 := h in
    let s3 := h* dFdx xn in 
    let s4 := 1.0 in 
    pair (pair s1 s2) (pair s3 s4). 
    
Definition leapfrog x0 v0 h F := 
    let v12 := snd (leapfrog_1 x0 v0 h F) in
    let x1  := fst (leapfrog_1 x0 v0 h F) in
    let v1  := v12 + 0.5* h* F x1 in
    pair x1 v1.

Cell evaluated. 
 

 
 
 Cell rolled back. 
 

 
 
 
 Rollback cell 
 

 
 
 Auto rollback

In [128]:
Definition id (x:R) := x.

Lemma leapfrog1_gt_v1:
     forall x0 v0 h,
     Rgt x0 0 -> Rgt v0 0 -> Rgt h 0 ->
     Rgt (fst (leapfrog_1 x0 v0 h id)) 0. 

Proof.
intros x0 v0 h Hx Hv Hh.
unfold leapfrog_1.
unfold fst. 
assert (H1 : Rlt 0 (h * v0)).
replace 0 with (h * 0).
apply (Rmult_lt_compat_l h 0 v0).
unfold Rgt in Hh; auto with real. 
unfold Rgt in Hv; auto with real.
auto with real.
unfold Rgt.
unfold Rgt in Hx.
assert (H2 : Rlt (x0 + 0) (x0 + h * v0)).
apply (Rplus_lt_compat_l x0 0 (h*v0)); auto with real.
(replace (x0 +0) with (x0) in H2); auto with real.
apply (Rlt_trans 0 x0 (x0 + h * v0)); auto with real.
Qed.

Lemma leapfrog_gt_v2:
     forall x0 v0 h,
     Rgt x0 0 -> Rgt v0 0 -> Rgt h 0 ->
     Rgt (fst (leapfrog_1 x0 v0 h id)) 0. 

Proof.
intros x0 v0 h Hx Hv Hh.
unfold leapfrog_1.
unfold fst. 
nra.
Qed.

Cell evaluated. 
 

 
 
 Cell rolled back. 
 

 
 
 
 Rollback cell 
 

 
 
 Auto rollback

The following definition of a symplectic map is taken from the paper [Why is the Boris alogrithm so good?](https://aip-scitation-org.proxy.library.cornell.edu/doi/pdf/10.1063/1.4818428), which clearly communicates the fact that symplectic maps are always volume preserving, but volume preserving maps are not necessarily symplectic. 

A one-step map $$\psi : \mathbf{z}_k \equiv (\mathbf{x}_k, \mathbf{v}_k) \rightarrow \mathbf{z}_{k+1} \equiv (\mathbf{x}_{k+1}, \mathbf{v}_{k+1})$$ is **symplectic** if its [Jacobian matrix](https://en.wikipedia.org/wiki/Jacobian_matrix_and_determinant) $$A = \frac{\partial \psi }{\partial \mathbf{z}_k}$$ is **symplectic**. Specifically, if $$A^T J A = J$$ where $$ J =  \begin{pmatrix}
0 & I \\
-I & 0 
\end{pmatrix}.$$ 

For a system of dimension $n$, we can write the block Jacobian with blocks of size $n \times n$, 

$$ A =  \begin{pmatrix}
S_1 & S_2 \\
S_3 & S_4
\end{pmatrix}.$$ 

We can therefore define a **symplectic** Jacobian as one with blocks that satisfy

$$S_1S_2^T = S_2S_1^T$$

$$S_3S_4^T = S_4S_3^T$$

$$S_1S_4^T - S_2S_3^T = I.$$

Given that we are working with at most three-dimensions, the above set of necessary and sufficient conditions for symplecticity on the block componenets of the Jacobian allow us to avoid dealing with more abstract definitions of matrices in Coq.

In the one-dimensional case, observe that area-preservation is a sufficient condition for symplecticity. More specifically, the determinant of the Jacobian of a one-dimensional symplectic map is 1. 

In [124]:
Definition is_symplectic_1d (jacobian:(R *R)*(R *R)) := 
    let S1 := fst(fst(jacobian)) in
    let S2 := snd(fst(jacobian)) in
    let S3 := fst(snd(jacobian)) in
    let S4 := snd(snd(jacobian)) in 
    S1*S4 - S2*S3 = 1.

Cell evaluated. 
 

 
 
 Cell rolled back. 
 

 
 
 
 Rollback cell 
 

 
 
 Auto rollback

In [217]:
Lemma leapfrog1_is_symplectic:
     forall x0 h,
     x0 = 0 -> Rgt h 0 ->
     is_symplectic_1d (jacob_leapfrog_1 x0 h id).
     
Proof. 
intros x0 h Hx Hh; unfold is_symplectic_1d; unfold jacob_leapfrog_1; unfold fst; unfold snd; unfold id; nra. 

Proving: leapfrog1_is_symplectic

No more subgoals